#### Import nécessaire

In [1]:
import os
from dotenv import load_dotenv
import requests
import pandas as pd
from datetime import datetime
import sqlite3

#### Déclaraton des variables 

In [2]:
load_dotenv()
api_key = os.getenv("API_KEY")
url = "https://api.openweathermap.org/data/2.5/weather"
villes = ["Paris", "Marseille", "Lyon", "Toulouse", "Nice", "Nantes", "Montpellier", "Strasbourg", "Bordeaux", "Lille", "Rennes", "Reims", "Le Havre", "Saint-Étienne", "Toulon", "Grenoble", "Dijon", "Angers", "Nîmes", "Villeurbanne"]
data = []


In [ ]:
print(api_key)

#### ***Avec Postman*** pour une meilleur visibilité ->  afficher un exemple de réponse en .json() pour voir comment est structuré le dictionnaire


In [ ]:
r = requests.get(f"http://api.openweathermap.org/data/2.5/weather?q=Paris&units=metric&appid={api_key}")
r.json()

#### On voit que une grande partie des informations dont nous avons besoin sont regroupés dans  "main", "wind" et "sys" avec comme valeur (Pour PARIS) : 
                        main : 
        "temp": 10.3, -> (Température actuelle)
        "feels_like": 8.8, ->  (Température ressentie)
        "temp_min": 9.5, -> (Température minimale)
        "temp_max": 11.25, -> (empérature maximale)
        "pressure": 1026, -> (Pression atmosphérique)
        "humidity": 54 -> (Humidité)

                        wind : 
        "speed": 7.72, -> (Vitesse du vent)
        "deg": 40 -> (Direction du vent)
        
                        sys : 
        "sunrise": 1680499491, -> (Lever du soleil)
        "sunset": 1680546168 -> (Coucher du soleil)

#### Creation du DataFrame regroupant les informations de météo par ville + conversion de certains données

In [5]:
for ville in villes:
    params = {
        "q": ville + ",FR",
        "appid": api_key,
        "units": "metric", 
        "lang": "fr"
    }
# utilisation de "units": "metric" ici pour convertir directement les unités de mesure (km/h et C°) pour faciliter la compréhension.

    reponse = requests.get(url, params=params)
    
    if reponse.status_code == 200:
        rep_json = reponse.json()
        
        meteo_france= {
            "Ville": ville,
            "Température actuelle C°": rep_json["main"]["temp"],
            "Température ressentie C°": rep_json["main"]["feels_like"],
            "Température minimale C°": rep_json["main"]["temp_min"],
            "Température maximale C°": rep_json["main"]["temp_max"],
            "Pression atmosphérique": rep_json["main"]["pressure"],
            "Humidité": rep_json["main"]["humidity"],
            "Vitesse du vent km/h": rep_json["wind"]["speed"],
            "Direction du vent en ° ": rep_json["wind"]["deg"],
            "Lever du soleil": datetime.fromtimestamp(rep_json["sys"]["sunrise"]).strftime('%Y-%m-%d %H:%M:%S'),
            "Coucher du soleil": datetime.fromtimestamp(rep_json["sys"]["sunset"]).strftime('%Y-%m-%d %H:%M:%S')
        }
# Utilisation de fromtimestamp() et strftime() ici pour prendre, dans un premier temps le Timestamp UNIX et renvoyer le "datetime" correspondant.
# Ensuite transormer le "datetime" dans le format souhaité pour une meilleur compréhension avec "strftime".
        data.append(meteo_france)
    else:
        print(reponse.status_code)

In [6]:
df_top20fr = pd.DataFrame(data)
# Pour définir Paris comme l'index1 (0 avant ce code)
df_top20fr = df_top20fr.reset_index()
df_top20fr.index += 1
df_top20fr = df_top20fr.drop(columns=['index'])

In [7]:
df_top20fr

,Ville,Température actuelle C°,Température ressentie C°,Température minimale C°,Température maximale C°,Pression atmosphérique,Humidité,Vitesse du vent km/h,Direction du vent en °,Lever du soleil,Coucher du soleil
1,Paris,11.55,10.09,10.62,12.70,1025,51,6.69,60,2023-04-03 07:24:51,2023-04-03 20:22:48
2,Marseille,17.87,16.63,15.08,19.26,1009,35,4.02,270,2023-04-03 07:16:58,2023-04-03 20:05:29
3,Lyon,8.27,7.37,6.04,11.08,1019,72,1.79,46,2023-04-03 07:18:42,2023-04-03 20:11:05
4,Toulouse,13.09,12.07,12.09,14.34,1020,62,5.66,310,2023-04-03 07:32:58,2023-04-03 20:21:56
5,Nice,10.51,9.13,9.34,11.00,1010,58,2.72,211,2023-04-03 07:09:27,2023-04-03 19:58:59
6,Nantes,7.96,5.20,7.81,7.96,1024,76,4.63,40,2023-04-03 07:42:06,2023-04-03 20:37:00
7,Montpellier,17.89,16.75,17.29,18.05,1013,39,6.69,360,2023-04-03 07:23:14,2023-04-03 20:12:11
8,Strasbourg,8.04,4.69,6.84,8.49,1025,49,6.17,60,2023-04-03 07:03:34,2023-04-03 20:00:56
9,Bordeaux,14.15,13.03,14.15,15.19,1021,54,3.60,50,2023-04-03 07:40:04,2023-04-03 20:31:01
10,Lille,11.32,9.68,8.85,12.75,1027,45,5.66,60,2023-04-03 07:20:16,2023-04-03 20:21:42


### Exporter le DF en CSV

In [8]:
df_top20fr.to_csv("meteo_france_top20.csv", index=False)


### Creation de Base de données SQL

In [9]:
connexion = sqlite3.connect("meteo_fr_top20.db")

connexion.execute("""
CREATE TABLE IF NOT EXISTS données_météo (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    Ville TEXT,
    Température_actuelle REAL,
    Température_ressentie REAL,
    Température_minimale REAL,
    Température_maximale REAL,
    Pression_atmosphérique INTEGER,
    Humidité INTEGER,
    Vitesse_du_vent REAL,
    Direction_du_vent INTEGER,
    Lever_du_soleil TEXT,
    Coucher_du_soleil TEXT
);
""")

for index, row in df_top20fr.iterrows():
    connexion.execute(
        """
        INSERT INTO données_météo (Ville, Température_actuelle, Température_ressentie, Température_minimale, Température_maximale,
        Pression_atmosphérique, Humidité, Vitesse_du_vent, Direction_du_vent, Lever_du_soleil, Coucher_du_soleil)
        VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
         """, 
    (row["Ville"], row["Température actuelle C°"], row["Température ressentie C°"], row["Température minimale C°"], row["Température maximale C°"], row["Pression atmosphérique"],
     row["Humidité"], row["Vitesse du vent km/h"], row["Direction du vent en ° "], row["Lever du soleil"], row["Coucher du soleil"]))

connexion.commit()
connexion.close()
